In [1]:
#Reading balanced dataset:
import pandas as pd

df = pd.read_excel('balanced_df.xlsx')
df.head()

,industrial_risk,management_risk,financial_flexibility,credibility,competitiveness,operating_risk,class
0,Medium,High,Low,Low,Low,Medium,bankruptcy
1,Low,High,Low,Low,Low,High,bankruptcy
2,High,Low,Low,Low,Low,High,bankruptcy
3,Medium,Low,Low,Medium,Low,High,bankruptcy
4,High,High,Low,Low,Low,High,bankruptcy


In [2]:
#Splitting data into features and target variable:
X = df.drop('class',axis = 1)
y = df['class']

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Create a ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('tnf', OneHotEncoder(), [0, 1, 2, 3, 4, 5])
    ],
    remainder='passthrough')
    
column_transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf', OneHotEncoder(), [0, 1, 2, 3, 4, 5])])

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

pipe = Pipeline(steps=[('Transformer',column_transformer),('Classifier',KNeighborsClassifier())])
pipe.fit(X, y)

Pipeline(steps=[('Transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tnf', OneHotEncoder(),
                                                  [0, 1, 2, 3, 4, 5])])),
                ('Classifier', KNeighborsClassifier())])

In [5]:
# Pipeline Working Test:
X_test_df = pd.DataFrame({
    'industrial_risk': ['Low'],
    'management_risk': ['Low'],
    'financial_flexibility': ['High'],
    'credibility': ['High'],
    'competitiveness': ['Medium'],
    'operating_risk': ['Low']
})

# Predict using the trained pipeline
y_pred = pipe.predict(X_test_df)

print("Predicted class labels:")
print(y_pred[0])

Predicted class labels:
non-bankruptcy


In [6]:
# Evaluate the model
accuracy = pipe.score(X, y)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.99


In [7]:
# Creating pickle file:

import pickle

pickle_out = open("model.pkl",mode = "wb")
pickle.dump(pipe,pickle_out)
pickle_out.close()

#### Streamlit App:

In [1]:
%%writefile bank.py

import streamlit as st
import pickle

# Load the trained model
pickle_in = open("model.pkl", "rb")
model = pickle.load(pickle_in)

@st.cache_data()
def calc(industrial_risk, management_risk, financial_flexibility, credibility, competitiveness, operating_risk):
    prediction = model.predict([[industrial_risk, management_risk, financial_flexibility, credibility, competitiveness, operating_risk]])
    return prediction

def main():
    st.title("Bankruptcy Prediction Application")
    st.image("image.jpg")

    # User input for risk factors
    st.sidebar.header("Input Risk Factors")
    industrial_risk = st.sidebar.radio('Industrial Risk', ('Low', 'Medium', 'High'))
    management_risk = st.sidebar.radio('Management Risk', ('Low', 'Medium', 'High'))
    financial_flexibility = st.sidebar.radio('Financial Flexibility', ('Low', 'Medium', 'High'))
    credibility = st.sidebar.radio('Credibility', ('Low', 'Medium', 'High'))
    competitiveness = st.sidebar.radio('Competitiveness', ('Low', 'Medium', 'High'))
    operating_risk = st.sidebar.radio('Operating Risk', ('Low', 'Medium', 'High'))

    # Predict Bankruptcy button
    if st.sidebar.button("Predict Bankruptcy"):
        prediction = calc(industrial_risk, management_risk, financial_flexibility, credibility, competitiveness, operating_risk)
        st.markdown(f'<div style="color: green; font-size: 18px;">Prediction : {prediction[0]}</div>', unsafe_allow_html=True)
        

if __name__ == "__main__":
    main()


Writing bank.py
